## CHECK  Calspec Name

- author : Sylvie Dagoret-Campagne
- creation date : 2025-12-10

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib.colors import LogNorm

from mpl_toolkits.axes_grid1 import make_axes_locatable

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.io import fits
from astropy.time import Time

In [ ]:
from matplotlib.dates import DateFormatter
#date_form = DateFormatter("%y-%m-%dT%H:%M")
date_form = DateFormatter("%y-%m-%d")

In [ ]:
from getCalspec.getCalspec import *

In [ ]:
#!ls *.csv

## READ file

In [ ]:
input_file = "2025-12-10_holosummary_all_filters_repo_main_20250107_20251209.csv"

In [ ]:
df = pd.read_csv(input_file,index_col=0)

In [ ]:
df

## List of target

In [ ]:
listOfTargets = df["target"].unique()

In [ ]:
all_unknown_targets = []
for target in listOfTargets:
    if not is_calspec(target):
        all_unknown_targets.append(target)   

In [ ]:
print("unknown CALSPEC target : ",all_unknown_targets)

In [ ]:
df["flag"] = df.apply(lambda x: x["target"] not in all_unknown_targets, axis=1)

In [ ]:
df_sel = df[~df["flag"]]

In [ ]:
#df_ser = df_sel.groupby(["day_obs","target"]).size()
ser = df_sel.groupby(["day_obs", "target"]).size()

In [ ]:
ser

In [ ]:
df = ser.reset_index(name="count")

In [ ]:
df

In [ ]:
# 1. Aggréger les counts par jour (si nécessaire)
# Si ton DataFrame a déjà une ligne par jour, passe directement à l'étape 2.
df_daily = df.groupby("day_obs")["count"].sum().reset_index()

# 2. Convertir day_obs en datetime
df_daily["day_obs"] = pd.to_datetime(df_daily["day_obs"], format="%Y%m%d")

# 3. Tracer le barplot
plt.figure(figsize=(12, 5))
plt.bar(df_daily["day_obs"], df_daily["count"], width=0.8, color="skyblue", edgecolor="black")

# Personnalisation
plt.title("Number of unknown target observations per night")
plt.xlabel("Date")
plt.ylabel("Number of observations")
plt.xticks(rotation=45)
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# 1. Convertir day_obs en datetime
df["day_obs"] = pd.to_datetime(df["day_obs"], format="%Y%m%d")

# 2. Pivoter le DataFrame pour avoir une colonne par target
df_pivot = df.pivot(index="day_obs", columns="target", values="count").fillna(0)

# 3. Tracer le barplot empilé
plt.figure(figsize=(12, 6))
ax = df_pivot.plot(kind="bar", stacked=True, figsize=(12, 6), color=plt.cm.tab20.colors)

# Personnalisation
plt.title("Number of observations per night (unknown target stacked)")
plt.xlabel("Date")
plt.ylabel("Number of observations")
plt.xticks(rotation=90)
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.legend(title="Unknown CALSPEC Target", bbox_to_anchor=(1.01, 1), loc="upper left")


# Ajustement de la légende et de la mise en page
plt.tight_layout()
plt.show()

